# Imports

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

# Request the website content and create bs object

In [3]:
r = requests.get("https://napieknewlosy.pl/pl/c/ODZYWKI-I-MASKI/13")
soup = bs(r.content, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lte IE 8]>     <html data-fmc="1" lang="pl" class="lt-ie8"> <![endif]-->
<!--[if IE 9]>         <html data-fmc="1" lang="pl" class="lt-ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 9]><!-->
<html data-fmc="1" data-pwa="1" lang="pl">
 <!--<![endif]-->
 <head data-ajaxbasket-mode="2" data-features="WyJwcm9kdWN0X2J1bmRsZXMiLCJ3ZWJkYXYiLCJzYWFzX2JpbGxpbmciLCJyZWRpcmVjdHNfY29uZmlnIiwicmVkaXJlY3RzX2RvbWFpbnMiLCJyZWRpcmVjdHNfb2JqZWN0cyIsInJlZGlyZWN0c19yb3V0ZXIiLCJyZWRpcmVjdHNfYXBpIiwiYWxsZWdyb19kb3dubG9hZF9vcmRlcnMiLCJhbGxlZ3JvX2ltcG9ydF9hdWN0aW9ucyIsImFsbGVncm9fdXBkYXRlX2F1Y3Rpb25zIiwic2VhcmNoX2luZGV4ZXIiLCJzZWFyY2hfYWRtaW4iLCJwd2EiLCJzZW9fZ2Z4IiwiYWxsZWdyb19zZW5kX3NoaXBwaW5nX251bWJlciIsInNob3dfcHJlbWl1bV9mZWF0dXJlcyIsInNlYXJjaF9jbGllbnQiLCJyZWNvbW1lbmRhdGlvbnNfdHJhY2tpbmciLCJiZWxsX21lc3NhZ2VzIiwicGF5bWVudF9ncGF5IiwibmJwX2N1cnJlbmNpZXMiLCJhbGxlZ3JvX2Rvd25sb2FkX29yZGVyc19zdXJjaGFyZ2VzIiwiYWxsZWdyb19hdXRvX2ltcG9ydF9hdWN0aW9ucyIsImFsbGVncm9fb3JkZXJfYW5kX3RyYW5zYWN0aW9uX3VpZF9m

# Web scraping code

In [4]:
peh_categories = {"E":"Emolientowe", "P":"Proteinowe", "H":"Humektantowe"}
products_by_peh = {"E": [], "P": [], "H": []}
base_url = "https://napieknewlosy.pl/"

def get_product_details(prod_link, peh_code):
    product_object = {}
    product_r =  requests.get(prod_link)
    product_soup = bs(product_r.content, 'lxml')

    inci_title = product_soup.find(text=re.compile(r".*Skład.*"))
    if inci_title is not None:
        ingredients = inci_title.parent.parent.text.split(':', 1)[1]
        if ingredients is None:
            return
        
        product_object["inci"] = ingredients                         
            
    else:
        return

    prod_title = product_soup.find('h1', {"class":"name"}).string
    prod_title_prep = prod_title.replace('\n', '').strip()
    product_object["title"]= prod_title_prep
    products_by_peh[peh_code].append(product_object)
    
def iterate_through_page_products(peh_soup, peh_code):
    products_list = peh_soup.find("div", {"class":"products viewphot s-row"})
    print("Started iterating through products")
    for product in products_list.find_all("div", {"class": "product s-grid-3 product-main-wrap"}):
        product_part_link = product.find("a").get("href")
        base_link = base_url
        product_link = base_link + product_part_link
        get_product_details(product_link, peh_code)
    print("Finished iterating through products")

    
def iterate_through_pagination(peh_soup, peh_code):
    page_soup = peh_soup
    while True:
        iterate_through_page_products(page_soup, peh_code)
        
        pag_ul = page_soup.find("ul", {"class":"paginator"})
        next_page_li = pag_ul.find("li", {"class":"last"})
        next_page_a = next_page_li.find("a", href=True)
        
        if next_page_a is None:
            break
        
        next_link_partial = next_page_a['href']
        next_link = base_url + next_link_partial
        page_r =  requests.get(next_link)
        page_soup = bs(page_r.content, 'lxml')
        
    
def handle_peh_links(peh_link, peh_code):
    peh_r =  requests.get(peh_link)
    peh_soup = bs(peh_r.content, 'lxml')
    pagination_list = peh_soup.find("ul", {"class":"paginator"})
    if pagination_list is not None:
        iterate_through_pagination(peh_soup, peh_code)
    else:
        iterate_through_page_products(peh_soup, peh_code)
    

for peh_code, peh_name in peh_categories.items():
    peh_partial_link = soup.find("a", text=peh_name)['href']
    peh_link = base_url+peh_partial_link
    handle_peh_links(peh_link, peh_code)
    
print(products_by_peh["P"][0])
print(products_by_peh["E"][0])
print(products_by_peh["H"][0])

Started iterating through products
Finished iterating through products
Started iterating through products
Finished iterating through products
Started iterating through products
Finished iterating through products
Started iterating through products
Finished iterating through products
Started iterating through products
Finished iterating through products
{'inci': '\xa0Aqua, Cetearyl Alcohol, Behentrimonium Chloride, Quaternium-91, Cetrimonium Methosulfate, Hydrolyzed Wheat Protein, Arctium Lappa Root Oil (olej łopianowy), Centaurium Erythraea Extract (centuria pospolita), Organic Acorus Calamus Root Extract (organiczny ekstrakt tataraku), Humulus Lupulus (Hops) Cone Extract (ekstrakt szyszek chmielowych), Organic Urtica Dioica (Nettle) Extract (organiczny ekstrakt pokrzywy), Robinia Pseudoacacia Flower Extract (ekstrakt kwiatów białej akacji), Guar Hydroxypropyltrimonium Chloride, Benzyl Alcohol, Benzoic Acid, Sorbic Acid, Parfum, Citric Acid, Beta Vulgaris (burak zwyczajny), Maltodextri

In [5]:
products_by_peh

{'E': [{'inci': '\xa0Aqua [Water],Cetyl alcohol,Behenamidopropyl dimethylamine,Butyrospermum parkii (Shea butter) butter,Sesamum indicum (Sesame) seed oil (*),Citrus aurantium dulcis (Orange) peel oil (*),Citrus limon (Lemon) peel oil (*),Citrus aurantium dulcis (Orange) flower extract (*),Calendula officinalis flower extract (*),Chamomilla recutita (Matricaria) flower extract (*),Citrus limon (Lemon) peel extract (*),Soy amino acids,Wheat amino acids,Arginine HCl,Serine,Threonine,Lactic acid,Sorbitol,Benzyl alcohol,Limonene,Sodium benzoate,Sodium dehydroacetate,Glycerin,Xanthan gum* z kontrolowanych upraw ekologicznych',
   'title': 'ALKEMILLA Balsamo Arancio e Limone Odżywka do włosów suchych, cienkich i farbowanych 250ml'},
  {'inci': '\xa0Aqua [Water],Cetyl alcohol,Behenamidopropyl dimethylamine,Butyrospermum parkii (Shea butter) butter,Helianthus annuus (Sunflower) seed oil (*),Citrus medica vulgaris peel oil,Foeniculum vulgare (Fennel) oil,Aloe barbadensis leaf extract (*),Malva 

In [10]:
products_by_peh["E"]

[{'inci': ['\xa0Aqua [Water]',
   'Cetyl alcohol',
   'Behenamidopropyl dimethylamine',
   'Butyrospermum parkii (Shea butter) butter',
   'Sesamum indicum (Sesame) seed oil (*)',
   'Citrus aurantium dulcis (Orange) peel oil (*)',
   'Citrus limon (Lemon) peel oil (*)',
   'Citrus aurantium dulcis (Orange) flower extract (*)',
   'Calendula officinalis flower extract (*)',
   'Chamomilla recutita (Matricaria) flower extract (*)',
   'Citrus limon (Lemon) peel extract (*)',
   'Soy amino acids',
   'Wheat amino acids',
   'Arginine HCl',
   'Serine',
   'Threonine',
   'Lactic acid',
   'Sorbitol',
   'Benzyl alcohol',
   'Limonene',
   'Sodium benzoate',
   'Sodium dehydroacetate',
   'Glycerin',
   'Xanthan gum* z kontrolowanych upraw ekologicznych'],
  'title': 'ALKEMILLA Balsamo Arancio e Limone Odżywka do włosów suchych, cienkich i farbowanych 250ml'},
 {'inci': ['\xa0Aqua [Water]',
   'Cetyl alcohol',
   'Behenamidopropyl dimethylamine',
   'Butyrospermum parkii (Shea butter) but

In [11]:
products_by_peh["P"]

[{'inci': ['\xa0Aqua',
   ' Cetearyl Alcohol',
   ' Behentrimonium Chloride',
   ' Quaternium-91',
   ' Cetrimonium Methosulfate',
   ' Hydrolyzed Wheat Protein',
   ' Arctium Lappa Root Oil (olej łopianowy)',
   ' Centaurium Erythraea Extract (centuria pospolita)',
   ' Organic Acorus Calamus Root Extract (organiczny ekstrakt tataraku)',
   ' Humulus Lupulus (Hops) Cone Extract (ekstrakt szyszek chmielowych)',
   ' Organic Urtica Dioica (Nettle) Extract (organiczny ekstrakt pokrzywy)',
   ' Robinia Pseudoacacia Flower Extract (ekstrakt kwiatów białej akacji)',
   ' Guar Hydroxypropyltrimonium Chloride',
   ' Benzyl Alcohol',
   ' Benzoic Acid',
   ' Sorbic Acid',
   ' Parfum',
   ' Citric Acid',
   ' Beta Vulgaris (burak zwyczajny)',
   ' Maltodextrin',
   ' Caramel.'],
  'title': 'AGAFIA Maska - balsam do włosów 20-minutowa, zioła syberyjskie 300ml'},
 {'inci': ['\xa0Aqua',
   ' Cetearyl Alcohol',
   ' Behentrimonium Chloride',
   ' Quaternium-91',
   ' Cetrimonium Methosulfate',
   

In [12]:
products_by_peh["H"]

[{'inci': ['\xa0Aqua',
   ' Cetearyl Alcohol',
   ' Behentrimonium Chloride',
   ' Starch Hydroxypropyltrimonium Chloride',
   ' Solanum Tuberosum (Potato) Starch (skrobia ziemniaczana)',
   ' Allium Cepa Extract (wyciąg z cebuli)',
   ' Saccharide Isomerate',
   ' Hydroxypropyltrimonium Hyaluronate',
   ' Aloe Barbadensis (Aloe Vera) Leaf Juice Powder (sok aloesowy)',
   ' Cetrimonium Chloride',
   ' Glycerin',
   ' Urea (mocznik)',
   ' Polysorbate 80',
   ' Hydroxypropyl Guar',
   ' Citric Acid',
   ' Sodium Citrate',
   ' Phenoxyethanol',
   ' Benzoic Acid',
   ' Dehydroacetic Acid',
   ' Caprylyl Glycol',
   ' Parfum',
   ' Amyl Cinnamal',
   ' Cinnamyl Alcohol',
   ' Hydroxycitronellal.'],
  'title': 'ANWEN Nawilżający Bez - MAŁA odżywka do włosów o różnej porowatości 100ml'},
 {'inci': ['\xa0Aqua',
   ' Cetearyl Alcohol',
   ' Behentrimonium Chloride',
   ' Starch Hydroxypropyltrimonium Chloride',
   ' Solanum Tuberosum (Potato) Starch (skrobia ziemniaczana)',
   ' Allium Cepa E

In [6]:
products_by_peh.keys()

dict_keys(['E', 'P', 'H'])

#  Convert data to df pandas object

In [7]:
df_e = pd.DataFrame(products_by_peh['E'])
df_e['peh_class'] = list(products_by_peh.keys())[0]
df_e

,inci,title,peh_class
0,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Arancio e Limone Odżywka do ...,E
1,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Cedro e Finocchio Odżywka do...,E
2,"Aqua [Water], Cetyl alcohol, Behenamidopropyl...",ALKEMILLA Balsamo Lavendo e Eucalipto Odżywka ...,E
3,"Aqua [Water] (Eau), Cetyl alcohol, Stearyl al...",ALKEMILLA K-HAIR Anti-Giallo Odżywka ochładzaj...,E
4,"Aqua [Water] (Eau), Stearyl alcohol, Behenami...",ALKEMILLA K-HAIR Multiwitaminowa maska do włos...,E
...,...,...,...
99,"Aqua, Cetearyl Alcohol, Coco-Caprylate/Caprat...",SESSIO Odżywka Vege z ekstraktem z mango 300g,E
100,"Cetearyl Alcohol, Theobroma Grandiflorum Seed...",SHAUSHKA Odżywka wegańska w kostce do włosów -...,E
101,"Aqua, Olea Europaea Fruit Oil, Cocos Nucifera...",TINKTURA Yellow Magic - Odżywka do włosów każd...,E
102,"Aqua, Cetyl Alcohol, Vitis Vinifera Seed Oil,...",VIANEK Intensywnie regenerująca maska do włosó...,E


In [8]:
df_p = pd.DataFrame(products_by_peh['P'])
df_p['peh_class'] = list(products_by_peh.keys())[1]
df_p

,inci,title,peh_class
0,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...","AGAFIA Maska - balsam do włosów 20-minutowa, z...",P
1,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...",AGAFIA Staroałtajska maska do włosów 300ml,P
2,"Aqua [Water] (Eau), Cetearyl alcohol, Behenam...","ALKEMILLA K-HAIR Maska do włosów normalnych, m...",P
3,"Aqua [Water] (Eau), Cetyl alcohol, Stearyl al...",ALKEMILLA K-HAIR Odżywka zwiększająca objętość...,P
4,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Odbudowująca maska do włosów suchych...,P
...,...,...,...
114,"Aqua (Water), Cetearyl Alcohol, Cocos Nucifer...",UMBERTO GIANNINI Banana Butter Vegan Leave-In ...,P
115,"Aqua, Cetyl Alcohol, Prunus Amygdalus Dulcis ...",VIANEK Intensywnie wzmacniająca maska z żywoko...,P
116,"Aqua, Cetearyl Alcohol, Behentrimonium Chlor...",YOPE Mini Odżywka Mleko Owsiane do włosów norm...,P
117,"Aqua, Cetearyl Alcohol, Behentrimonium Chlor...",YOPE Odżywka Mleko Owsiane do włosów normalnyc...,P


In [9]:
df_h = pd.DataFrame(products_by_peh['H'])
df_h['peh_class'] = list(products_by_peh.keys())[2]
df_h

,inci,title,peh_class
0,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...",ANWEN Nawilżający Bez - MAŁA odżywka do włosów...,H
1,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...",ANWEN Nawilżający Bez - odżywka do włosów o ró...,H
2,"Aqua/Water/Eau, Aloe Barbadensis Leaf Juice, ...",AS I AM DRY & ITCHY Olive & Tea Tree Oil Condi...,H
3,"Aqua/Water/Eau, Glycerin, Aloe Barbadensis Le...",AS I AM DRY & ITCHY Olive & Tea Tree Oil Leave...,H
4,"Aqua/Water/Eau, Glycerin, Aloe Barbadensis Le...",AS I AM So Much Moisture! Nawilżająca odżywka ...,H
5,"Aqua/Water, Glycerin, Stearyl Alcohol, Polygl...",BANDI Tricho-maska wzmacniająca do skóry głowy...,H
6,"Aqua/Water, Glycerin, Stearyl Alcohol, Polygl...",BANDI Tricho-maska wzmacniająca do skóry głowy...,H
7,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...",BASICLAB DERMOCOSMETICS CAPILLUS Odżywka do wł...,H
8,"Aqua, Cetearyl Alcohol, Behentrimonium Chlori...",BASICLAB DERMOCOSMETICS CAPILLUS Odżywka do wł...,H
9,"Nałożyć odżywkę na umyte, wilgotne włosy. Zmy...",BIOLAVEN Odżywka z olejem z pestek winogron 300ml,H


In [10]:
frames = [df_e, df_p, df_h]

df_npk_final = pd.concat(frames)
df_npk_final

,inci,title,peh_class
0,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Arancio e Limone Odżywka do ...,E
1,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Cedro e Finocchio Odżywka do...,E
2,"Aqua [Water], Cetyl alcohol, Behenamidopropyl...",ALKEMILLA Balsamo Lavendo e Eucalipto Odżywka ...,E
3,"Aqua [Water] (Eau), Cetyl alcohol, Stearyl al...",ALKEMILLA K-HAIR Anti-Giallo Odżywka ochładzaj...,E
4,"Aqua [Water] (Eau), Stearyl alcohol, Behenami...",ALKEMILLA K-HAIR Multiwitaminowa maska do włos...,E
...,...,...,...
52,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów osłabionych z te...,H
53,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów osłabionych zabi...,H
54,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów suchych i matowy...,H
55,"Aqua, Cetearyl Alcohol, Behenamidopropyl Dime...",VIS PLANTIS Odżywka do włosów z tendencją do p...,H


# Save and check dataset

In [11]:
path_to_output = r'C:\Users\Pszczółka\Projects\Data_Science_Projects\PEH_Classifier\Data_collection'
df_npk_final.to_csv(path_to_output + r'\Hair_cosmetics_by_PEH_napieknewlosy.csv')

In [12]:
df_npk_final_test = pd.read_csv('Hair_cosmetics_by_PEH_napieknewlosy.csv')
df_npk_final_test

,Unnamed: 0,inci,title,peh_class
0,0,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Arancio e Limone Odżywka do ...,E
1,1,"Aqua [Water],Cetyl alcohol,Behenamidopropyl d...",ALKEMILLA Balsamo Cedro e Finocchio Odżywka do...,E
2,2,"Aqua [Water], Cetyl alcohol, Behenamidopropyl...",ALKEMILLA Balsamo Lavendo e Eucalipto Odżywka ...,E
3,3,"Aqua [Water] (Eau), Cetyl alcohol, Stearyl al...",ALKEMILLA K-HAIR Anti-Giallo Odżywka ochładzaj...,E
4,4,"Aqua [Water] (Eau), Stearyl alcohol, Behenami...",ALKEMILLA K-HAIR Multiwitaminowa maska do włos...,E
...,...,...,...,...
275,52,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów osłabionych z te...,H
276,53,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów osłabionych zabi...,H
277,54,"Aqua, Cetearyl Alcohol, Glycerin, Behenamidop...",VIS PLANTIS Odżywka do włosów suchych i matowy...,H
278,55,"Aqua, Cetearyl Alcohol, Behenamidopropyl Dime...",VIS PLANTIS Odżywka do włosów z tendencją do p...,H
